In [17]:
# -*- coding: utf-8 -*-

import os
import httplib2
import sys
import google.oauth2.credentials
import google_auth_oauthlib.flow

from googleapiclient.discovery import build, build_from_document
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from apiclient.discovery import build
from oauth2client.tools import argparser, run_flow
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage


import pandas as pd
import downloadPermission

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import re

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pickle

from nltk.tokenize import word_tokenize # or use some other tokenizer
from nltk.corpus import stopwords
import string

In [2]:
dbname = 'youtubeSpon'
username = 'april' # change this to your username

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
con = None
con = psycopg2.connect(database = dbname, user = username)
# query:
sql_query = """
SELECT * FROM "youtubeEmmy" WHERE sponsored <> 'None';
"""
youtube_data_from_sql = pd.read_sql_query(sql_query,con)
youtube_data_from_sql.tail()

,index,channel,title,video_id,description,captions,sponsored,fullCaptions
253,810,emmymadeinjapan,WINNERS Curry Set Giveaway,CQTp_Hh8qH4,Announcing the winners of the Popin Cookin' Ka...,"1\n00:00:06,680 --> 00:00:11,519\nhi everyone ...",no,grins uglies hides and we welcome back to anot...
254,811,emmymadeinjapan,PEZ giveaway WINNERS,stpWXStiw4k,Watch to find out if you're one of the lucky w...,"1\n00:00:08,540 --> 00:00:13,200\nhello everyb...",no,hello everybody I'm back again and it's it's s...
255,814,emmymadeinjapan,PEZ Whatcha Eating #82,yJfm6wFnr2s,[GIVEAWAY CLOSED] Eating old fashioned PEZ can...,"1\n00:00:08,470 --> 00:00:15,980\nhello everyb...",no,greetings my lovelies hi and welcome back to a...
256,815,emmymadeinjapan,Gummy Burger Set - Lunch Bag: Whatcha Eating? #81,9cbb_kdUspU,[GIVEAWAY CLOSED] A miniature gummy version of...,"1\n00:00:07,910 --> 00:00:14,639\nhi everybody...",no,everybody it's a me and I'm back and I'm still...
257,836,emmymadeinjapan,Giveaway Winners - Chocolat de Tomato,R-VvqR0EFNk,Wanna try Japanese tomato chocolate for yourse...,"1\n00:00:07,280 --> 00:00:12,870\nhello you tw...",no,frooty frooty frooty frooty frooty frooty brin...


In [82]:
data_with_captions= pd.read_pickle("emmy.pickle")

In [3]:
import random
import nltk

In [4]:
sample_size = 15
not_spon = [index for index, item in enumerate(youtube_data_from_sql.sponsored) if item == 'no']
not_spon_sample = random.sample(not_spon,sample_size)
spon = [index for index, item in enumerate(youtube_data_from_sql.sponsored) if item == 'yes']
spon_sample = random.sample(spon,sample_size)

In [5]:
train = youtube_data_from_sql.iloc[not_spon_sample + spon_sample, :]
train

,index,channel,title,video_id,description,captions,sponsored,fullCaptions
3,4,emmymadeinjapan,CHECKERBOARD Pan TEST Gatorade Cake Recipe | D...,SsFO-NvMcNs,This set of pans promises to make a triple lay...,"1\n00:00:06,100 --> 00:00:09,179\nGreetings my...",no,Happy Holidays lovelies hi it's Emmy and welco...
121,139,emmymadeinjapan,GLUE GUN & CHEESE | DIY Fondoodler | extrude m...,2w5W1t4gOC0,"Today we're answering the age old question, Wh...","1\n00:00:04,529 --> 00:00:09,210\ncreated my l...",no,greetings YouTube hi it's Emmy I'm back this t...
53,68,emmymadeinjapan,EGGSTRACTOR egg peeling gadget | Does it Work?,jvYxlbIGlxw,It's supposed to simplify your life by making ...,"1\n00:00:06,220 --> 00:00:09,892\nGreetings my...",no,"Greetings my lovelies! Hi, it's Emmy -- welcom..."
64,80,emmymadeinjapan,OMATSURI POPIN' COOKIN' Japanese Candy Kit mak...,356nKRf9lBA,"Summers in Japan means matsuri, summertime fes...","1\n00:00:04,160 --> 00:00:08,730\ngreetings my...",no,greetings uglies hi it's Emmy welcome back to ...
250,706,emmymadeinjapan,Emmy Eats China - tasting Chinese sweets,mmsAWGJmF-E,"Sampling a bit of the China, including some Mi...","1\n00:00:07,490 --> 00:00:14,219\nhi lovelies ...",no,"Greetings my lovelies! Hi, it's Emmy, and welc..."
106,124,emmymadeinjapan,Trader Joe's KNOCK-OFF Candy Bar TASTE TEST | ...,MK_wNXdIDd8,Trader Joe's has a couple of candy bars that s...,"1\n00:00:01,740 --> 00:00:04,930\n[Music]\n\n2...",no,[A Mission music] Greetings my beautiful lovel...
41,53,emmymadeinjapan,Cooking a Whole BLACK Silkie CHICKEN in a PUMP...,_nxFjhcyTMQ,Black Silkie Chinese herbal chicken soup is a ...,"1\n00:00:00,000 --> 00:00:05,560\n[Music]\n\n2...",no,[Music] greetings my lovelies hi it's Emmy wel...
234,254,emmymadeinjapan,Japanese OREO Taste Test - Whatcha Eating?,lf5aDczijKQ,Tasting Japanese Oreo varieties on this Emmyma...,"1\n00:00:06,220 --> 00:00:07,360\nHello, my be...",no,greetings my lovelies hi it's I mean welcome b...
35,47,emmymadeinjapan,Kitty Litter CAKE | Dollar $tore Recipe,M3ooQjM2OUA,This kitty litter cake's repulsiveness is only...,"1\n00:00:01,860 --> 00:00:05,439\n[Music]\n\n2...",no,Grimm's always hides I mean welcome back to te...
127,144,emmymadeinjapan,BIGMAC VS WHOPPER | Versus BURGER edition,UYNfrhspW0A,A classic burger battle: McDonald's Big Mac v....,"1\n00:00:00,660 --> 00:00:08,160\n[Music]\n\n2...",no,[Music] greetings my lovelies hi and we welcom...


In [6]:

stop = stopwords.words('english') + list(string.punctuation)

In [7]:
train_tokens = []
for sent in train.fullCaptions:
    train_tokens.append([i for i in word_tokenize(sent.lower()) if i not in stop])

In [8]:
words = [y for x in train_tokens for y in x]

In [9]:
all_words = nltk.FreqDist(w.lower() for w in words)
word_features = list(all_words)



In [50]:
word_features

['happy',
 'holidays',
 'lovelies',
 'hi',
 "'s",
 'emmy',
 'welcome',
 'back',
 'another',
 'episode',
 'thirsty',
 'today',
 'couple',
 'sodas',
 "'m",
 'going',
 'try',
 'sent',
 'company',
 'called',
 'hot',
 'lips',
 'boo',
 'based',
 'portland',
 'oregon',
 'give',
 'opinion',
 'thanks',
 'outlets',
 'maryam/mary',
 'sounds',
 'delicious',
 'looks',
 'lot',
 'like',
 'blackberry',
 'raspberry',
 'well',
 'deliciously',
 'refreshing',
 'bottle',
 'begins',
 'fresh',
 'picked',
 'fruit',
 'peak',
 'family',
 'farms',
 'pacific',
 'northwest',
 'bright',
 'juicy',
 'all-natural',
 'raspberries',
 'flecked',
 'bits',
 'sweet',
 'pulp',
 'paired',
 'soft',
 'smooth',
 'bubbles',
 'good',
 'bottles',
 'chilled',
 'also',
 'ice',
 'soda',
 'ingredients',
 'marionberry',
 'sandy',
 'boring',
 'cane',
 'sugar',
 'organic',
 'lawn',
 'lemon',
 'juice',
 "n't",
 'mean',
 'insult',
 'manners',
 'really',
 'organ',
 'si',
 'alright',
 'let',
 'think',
 'type',
 'kind',
 'hello',
 'postal',
 '

In [51]:
filename = 'dictionary.sav'
pickle.dump(word_features, open(filename, 'wb'))

In [ ]:
def document_features(document):
    document_words = document.split()
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [10]:
feature_set = []
for i in range(len(train)):
    feature_set.append((document_features(train.fullCaptions.iloc[i]), 
           train.sponsored.iloc[i]))

In [11]:
classifier = nltk.NaiveBayesClassifier.train(feature_set)

In [12]:
classifier.show_most_informative_features(20)

Most Informative Features
         contains(thick) = True              yes : no     =      5.0 : 1.0
          contains(hope) = False             yes : no     =      5.0 : 1.0
            contains(ah) = True              yes : no     =      4.3 : 1.0
          contains(thin) = True              yes : no     =      4.3 : 1.0
      contains(together) = True              yes : no     =      3.7 : 1.0
         contains(whole) = True              yes : no     =      3.7 : 1.0
          contains(bits) = True               no : yes    =      3.7 : 1.0
      contains(remember) = True              yes : no     =      3.7 : 1.0
          contains(came) = True              yes : no     =      3.7 : 1.0
       contains(coconut) = True              yes : no     =      3.7 : 1.0
           contains(pop) = True               no : yes    =      3.7 : 1.0
          contains(case) = True              yes : no     =      3.7 : 1.0
       contains(cookies) = True              yes : no     =      3.7 : 1.0

In [13]:
train.description.iloc[['mint'in cap for cap in train.captions]]

169    On my last trip to California I stopped into C...
188    A big thank you to Progresso for sponsoring th...
101    Visit https://squarespace.com/emmy and enter t...
Name: description, dtype: object

In [55]:
spon_test = list(set(spon) - set(spon_sample))
not_spon_test =  random.sample(list(set(not_spon) - set(not_spon_sample)),len(spon_test))

test = youtube_data_from_sql.iloc[not_spon_test + spon_test, :]

test_feature_set = []
for i in range(len(test)):
    test_feature_set.append((document_features(test.fullCaptions.iloc[i]), 
           test.sponsored.iloc[i]))
    
    
nltk.classify.accuracy(classifier, test_feature_set)

0.75

In [52]:
test

,index,channel,title,video_id,description,captions,sponsored,fullCaptions
148,166,emmymadeinjapan,SPICY CHEESE RAMEN CHALLENGE,tFo-ZFxy_Tc,I found a cheesy version of the Samyang noodle...,"1\n00:00:00,000 --> 00:00:03,720\n[Music]\n\n2...",no,greetings uh plays hi it's Emmy welcome back t...
108,126,emmymadeinjapan,WAFFLE HOT DOG Recipe - Weenies,CrPXJZikzCM,Here's how to take breakfast to the next level...,"1\n00:00:00,000 --> 00:00:02,360\n[Music]\n\n2...",no,everybody it's me I'm back back in with a how ...
105,123,emmymadeinjapan,$1 CHICKEN FRIED STEAK & homemade gravy | DOLL...,RNQdkP0uUQc,Visit https://squarespace.com/emmy and enter t...,"1\n00:00:00,320 --> 00:00:07,350\n[Music]\n\n2...",yes,Greetings my lovelies~ Hi it's Emmy. Welcome b...
14,15,emmymadeinjapan,Emmy Takes the SUPERTASTER Test - counting tas...,P_8ZYxJ8ay0,Check out HELLO FRESH: http://bit.ly/2fiQBC0 a...,"1\n00:00:05,200 --> 00:00:10,200\nGreetings my...",yes,you everyone I'm back it's been a while I've n...


In [58]:
i=2
(classifier.classify(document_features(test.fullCaptions.iloc[i])), test.sponsored.iloc[i])

('yes', 'yes')

In [18]:
# save the model to disk
filename = 'sponView_classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [64]:
filename = 'sponView_classifier.sav'
classifier = pickle.load(open(filename, 'rb'))
nltk.classify.accuracy(classifier, test_feature_set)

0.75

In [65]:
for i in range(4):
    print((classifier.classify(document_features(test.fullCaptions.iloc[i])), 
           test.sponsored.iloc[i],test.video_id.iloc[i]) )

('yes', 'no', 'gZe7VETOgaY')
('no', 'no', 'LL9AG9U7Vqw')
('yes', 'yes', 'RNQdkP0uUQc')
('yes', 'yes', 'P_8ZYxJ8ay0')


# flask code

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import re

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pickle

from nltk.tokenize import word_tokenize # or use some other tokenizer
from nltk.corpus import stopwords
import string

In [81]:
#### given video id pull captions
######input video_id
def get_captions(video_id):
    ###### for MVP just take from table. for next week figure out authentication and implement parsing
    dbname = 'youtubeSpon'
    username = 'april' # change this to your username

    engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
    con = None
    con = psycopg2.connect(database = dbname, user = username)

    # query:
    sql_query = """SELECT * FROM "youtubeEmmy" WHERE "youtubeEmmy".video_id = '%s';"""%video_id
    selectedCaption = pd.read_sql_query(sql_query,con)
    return selectedCaption

get_captions('P_8ZYxJ8ay0').fullCaptions

0    you everyone I'm back it's been a while I've n...
Name: fullCaptions, dtype: object

In [ ]:
def document_features(document):
    document_words = document.split()
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [49]:
#### run classifier
def run_classifier(video_id):
    #load model
    filename = 'sponView_classifier.sav'
    classifier = pickle.load(open(filename, 'rb'))
    captions = get_captions(video_id)
    return classifier.classify(document_features(captions))
    
run_classifier('QDlbOrOO0_E')

'no'

In [53]:
run_classifier('RNQdkP0uUQc')

'no'